In [1]:
import json
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import os
from pycocotools import coco
import xml.etree.ElementTree as ET
import xml
from pycocotools import mask as comask
import cv2
from utils import DateCaptured, coco_bbox_creator, xml_retreiver, iou_calculator, biggest_segment, mask2poly, binary_mask_to_rle
# import csv ozan changed

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### CATEGORY INFORMATION

# Add the name of the folders inside the dataset and
# assign some label id. Start iding from 91.

dict_of_categories = {'face': 91}

In [3]:
### DATASET INFORMATION

# Enter the original/custom JSON file to be used.
# I used the original train2014 JSON file containing
# the instances.

json_dir = "Datasets/coco/annotations_old/instances_train2014.json"

# for the purpose of creating validation images json
# json_dir = "Datasets/coco/annotations_old/instances_val2014.json" 

# for the purpose of creating testing images json
# json_dir = "Datasets/coco/annotations_old/instances_minival2014.json" 

### For viewing the content of the new json file. ###
# json_dir = "Datasets/coco/annotations/instances_train2014.json"

### minival2014 will become the main dictionary ###
minival2014 = json.load(open(json_dir))

In [4]:
### CLEARING THE DATASET
# Use that if you only going to make a dataset of your own.
# If you are going to append the images, this operation is not necessary.

minival2014['images'] = []
minival2014['annotations'] = []
# minival2014['categories'] = []

In [5]:
# Viewing the number of instances per class and gathering some statistics about them.
# labels = np.zeros((91, ), dtype=np.int32)
# for annotation in minival2014['annotations']:
#     labels[annotation['category_id']] += 1
# used_label_ids = np.where(labels!=0)
# ex_labels = labels[used_label_ids]
# print(ex_labels)
# print("Average instances per label: {}".format(sum(ex_labels)/81))
# print("Maximum instances per label: {}".format(max(ex_labels)))
# print("Minimum instances per label: {}".format(min(ex_labels)))

In [6]:
### APPENDING CLASSES TO THE CATEGORIES

### For viewing the categories inside the JSON file.
# minival2014['categories']

# Add the classes based on to the dict_of_categories
# Below shows an example for adding the face class to the main dictionary. 

minival2014['categories'].append({'id': 91, 'name': 'face', 'supercategory': 'person'})



In [7]:
# Some functions to be used:
def isjpg(string):
    if string[-4:]==".jpg":
        return True 
    
def ispng(string):
    if string[-4:]==".png":
        return True
    
def save_bboxes(image_name, bbox_coor):
    with open('img_bbox.txt','a+') as file:
        file.write(str(image_name)+','+str(bbox_coor[0])+','+str(bbox_coor[1])+','+str(bbox_coor[2])+','+str(bbox_coor[3])+'\n')

        

In [9]:
# LabelMe type
dataset_dir = "/home/mspr/Desktop/Mask_RCNN/Datasets/"  # Will be changed with argparse, includes directories for video names and corresponding frames inside
# dataset_name = "MSPR_Dataset_valset/"
dataset_name = "MSPR_Dataset_20July/"
new_masks_dir = "Masks_GT/"
modify_mask_gt = True
area_thr = 60
objective = "train"
# objective = "valid"
datasettype_imported = "labelme"

# coco_dir = os.path.join(dataset_dir, "coco/val2014/")
coco_dir = os.path.join(dataset_dir, "coco/train2014/")

##########################################################################
##########################################################################
##########################
# TO DO, MATCHING AND CALCULATING IOU'S BETWEEN OZAN'S AND FILIZ'S ANNOTATIONS
##########################
avg_iou = []

##########################
# ANNOTATIONS OF FILIZ   #
##########################
filiz_annotations = os.path.join("/home/mspr/Desktop/Mask_RCNN/Datasets/MSPR_Dataset/face/filiz_Annotations",
                                 "pascal_voc_face annotation.txt")

# image_nrs, bbox_filiz = txt_bbox_parser(filiz_annotations)
##########################################################################
##########################################################################

# Added image and annotation ids will start from 600k. 
image_id = 600000
annotation_id = 600000
if datasettype_imported == "labelme":
    for class_name in dict_of_categories.keys():
        annot_dir = os.path.join(dataset_dir, dataset_name, class_name, "Annotations/")
        mask_dir = os.path.join(dataset_dir, dataset_name, class_name, "Masks/")
        image_dir = os.path.join(dataset_dir, dataset_name, class_name, "Images/") 

        annot_all_files = sorted(os.listdir(os.path.join(dataset_dir, dataset_name, class_name, annot_dir)))
        for xml_file in annot_all_files:

            tree = ET.parse(os.path.join(annot_dir, xml_file))
            root = tree.getroot()

            delete_flags = root.findall("./object/deleted")
            sum_deleted = sum(map(int, [delete_flag.text for delete_flag in delete_flags]))

            xmins = root.findall("./object/segm/box/xmin")
            ymins = root.findall("./object/segm/box/ymin")
            xmaxs = root.findall("./object/segm/box/xmax")
            ymaxs = root.findall("./object/segm/box/ymax")

            mask_filenames = root.findall("./object/segm/mask")

            # Not yet polygon type of annotations are supported.
            assert root.findall("./object/polygon") == [], "Polygon type of annotations are not yet to be supported."

            # IMAGES
            # Image will be copied to MS COCO train directory and file name will be hold

            image_filename = xml_retreiver(root, 'filename')

            assert image_filename[-4:] == ".jpg", "The filename extension is not .jpg. Name of file: {}".format(image_filename)
            print(xml_file, end=" ")

            image = io.imread(os.path.join(image_dir, image_filename))

            if objective == "train":
                if image_id<1000000:
                    image_target_filename = "COCO_train2014_000000"+str(image_id)+".jpg"
                elif (image_id>=1000000 or image_id<=10000000):
                    image_target_filename = "COCO_train2014_00000"+str(image_id)+".jpg"
            elif objective == "valid":
                if image_id<1000000:
                    image_target_filename = "COCO_val2014_000000"+str(image_id)+".jpg"
                elif (image_id>=1000000 or image_id<=10000000):
                    image_target_filename = "COCO_val2014_00000"+str(image_id)+".jpg"


            io.imsave(os.path.join(coco_dir, image_target_filename), image)

            # Image height and width
            height = int(xml_retreiver(root, 'nrows'))
            width = int(xml_retreiver(root, 'ncols'))

            assert height == image.shape[0]
            assert width == image.shape[1]


            # Miscellaneous metadata
            date_captured = DateCaptured()
            coco_url = 'http://www.mspr.itu.edu.tr/'
            flickr_url = 'http://www.mspr.itu.edu.tr/'
            license = np.random.randint(8)

            # Appending to 'images'
            minival2014['images'].append({'coco_url': coco_url, 'file_name': image_target_filename, 
                                         'date_captured': date_captured, 'flickr_url': flickr_url,
                                         'height': height, 'id': image_id, 'license': license, 
                                         'width': width})
            # Polygon Handling

            for d, delete_flag in enumerate(delete_flags):
                delete_flag = int(delete_flag.text)
                if not delete_flag:

                    # ANNOTATIONS
                    # Bbox
                    bbox_x = int(xmins[d].text)
                    bbox_y = int(ymins[d].text)
                    bbox_w = int(xmaxs[d].text) - bbox_x
                    bbox_h = int(ymaxs[d].text) - bbox_y
                    bbox_list = [bbox_x, bbox_y, bbox_w, bbox_h]
                    save_bboxes(image_filename[:-4], bbox_list)

    ##########################################################################
    ##########################################################################
    #                 queries_findid = [d for d, image_nr in enumerate(image_nrs) if image_nr==image_filename[:-4]]
    #                 for query in queries_findid:
    #                     print(bbox_filiz[query])
    #                     print(bbox_list)
    #                 iou_for_each_xml_object = [iou_calculator(bbox_filiz[query], bbox_list) for query in queries_findid]
    #                 avg_iou.append(iou_calculator(bbox_filiz[query], bbox_list) for query in queries_findid)    
    #                 print(iou_for_each_xml_object)
    #                 print("")


    #                 print(bbox_x, bbox_y, bbox_w, bbox_h)

    #                 with open("bbox_info.txt", "a+") as f:
    #                     f.write(image_filename[:-4]+","+str(bbox_x)+","+str(bbox_y)+","+
    #                             str(bbox_x+bbox_w)+","+str(bbox_y)+","+
    #                             str(bbox_x)+","+str(bbox_y+bbox_h)+","+
    #                            str(bbox_x+bbox_w)+","+str(bbox_y+bbox_h)+"\n")

    ##########################################################################
    ##########################################################################

                    # Class
                    category_id = dict_of_categories[class_name]

                    # Masks Segmentation
                    mask = cv2.imread(os.path.join(mask_dir, mask_filenames[d].text), 0)
                    polygons, binary_image = mask2poly(mask, morph=modify_mask_gt)

                    # Mask should be defined with at least 60 points.
                    area_binary_img = len(np.where(binary_image==1)[0])
                    if area_binary_img < area_thr:
                        continue

                    # Write the new modified masks to a folder
                    if modify_mask_gt:
                        new_mark_dir = os.path.join(dataset_dir, dataset_name, class_name, new_masks_dir)
                        cv2.imwrite(os.path.join(new_mark_dir, mask_filenames[d].text), binary_image*255)

                    # Generate the RLE version of mask
                    RLE_mask = binary_mask_to_rle(mask)
                    RLE_mask_coco = comask.encode(np.asfortranarray(binary_image.astype(np.uint8)))

                    # Area
                    area = comask.area(RLE_mask_coco)

                    #  Iscrowd
                    if (len(mask_filenames) - sum_deleted) > 1:
                        iscrowd = 1
                        minival2014['annotations'].append({'iscrowd': iscrowd, 'bbox': bbox_list, 'id': annotation_id,
                                                           'image_id': image_id, 'segmentation': RLE_mask,
                                                           'area': area, 'category_id': dict_of_categories[class_name]})
                    else:
                        iscrowd = 0
                        minival2014['annotations'].append({'iscrowd': iscrowd, 'bbox': bbox_list, 'id': annotation_id,
                                                      'image_id': image_id, 'segmentation': polygons,
                                                      'area': area, 'category_id': dict_of_categories[class_name]})

                    # Appending into annotations

                    annotation_id+=1
            image_id+=1

                
#                 print(bbox_x, bbox_y, bbox_w, bbox_h)
# print("Average IoU = {}".format(sum(avg_iou)/len(avg_iou)))

000001.xml 000021.xml 000025.xml 

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


000027.xml 000030.xml 000035.xml 000038.xml 000043.xml 000050.xml 000051.xml 000058.xml 000069.xml 000073.xml 000076.xml 000081.xml 000085.xml 000089.xml 000090.xml 000096.xml 000101.xml 000104.xml 000105.xml 000113.xml 000124.xml 000125.xml 000127.xml 000129.xml 000133.xml 000138.xml 000139.xml 000144.xml 000146.xml 000151.xml 000159.xml 000162.xml 000164.xml 000165.xml 000166.xml 000169.xml 000170.xml 000171.xml 000173.xml 000174.xml 000177.xml 000181.xml 000182.xml 000191.xml 000192.xml 000193.xml 000200.xml 000202.xml 000205.xml 000206.xml 000212.xml 000220.xml 000222.xml 000229.xml 000230.xml 000231.xml 000237.xml 000245.xml 000247.xml 000248.xml 000252.xml 000257.xml 000258.xml 000259.xml 000264.xml 000265.xml 000269.xml 000271.xml 000272.xml 000275.xml 000276.xml 000278.xml 000280.xml 000282.xml 000283.xml 000285.xml 000286.xml 000287.xml 000297.xml 000298.xml 000299.xml 000302.xml 000305.xml 000308.xml 000310.xml 000315.xml 000319.xml 000320.xml 000321.xml 000322.xml 000323.xml

In [10]:
# Storing the data in memory and then writes them into a file. 

# from decimal import Decimal
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

with open("Datasets/coco/annotations/instances_train2014.json","w") as f:
    
# with open("Datasets/coco/annotations/instances_minival2014.json","w") as f:
    data = json.dumps(minival2014, cls=MyEncoder, indent=4)
    f.write(data)
